In [1]:
import io
import os
import glob
import locale
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/Users/edupmon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [3]:
pd.set_option('display.float_format',
              lambda x: locale.format_string('%.2f', x, grouping=True))

# Análise da exportação brasileira de vinhos de mesa

## Períodos de interesse (entre 1970 e 2023)

In [4]:
ano_inicial = 2009
ano_final = 2023

## Obtendo os dados de exportação de vinhos

## Carregando os dados de exportação de vinhos em um dataframe

In [5]:
# Obtendo a lista de arquivos que será carregada no dataframe
arqs_csv = glob.glob(os.path.join('dados', 'exp-vinho-*.csv'))

# Definindo o dataframe que irá receber os dados de todos os arquivos CSV
'''Concatenação dos dataframes obtidos à partir da lista de arquivos (list
   comprehension abaixo), ignorando a primeira linha dos CSVs e definindo o
   nome das colunas manualmente'''
exportacao = pd.concat([pd.read_csv(arq,
                                    skiprows=1,
                                    na_values=0,
                                    names=['pais',
                                           'qtd_litros',
                                           'vlr_usd',
                                           'ano']) for arq in arqs_csv],
                       ignore_index=True)

exportacao.fillna(0, inplace=True)
exportacao.head()

,pais,qtd_litros,vlr_usd,ano
0,Afeganistão,"0,00","0,00",2021
1,África do Sul,"0,00","0,00",2021
2,"Alemanha, República Democrática","225.086,00","393.482,00",2021
3,Angola,"54.786,00","84.235,00",2021
4,Anguilla,"0,00","0,00",2021


## Analisando a exportação total por país

In [6]:
# Exportação total por país
total_por_pais = exportacao.groupby('pais')[['qtd_litros', 'vlr_usd']]\
                           .sum()\
                           .copy()

total_por_pais.head()

,qtd_litros,vlr_usd
pais,,
Afeganistão,"0,00","0,00"
"Alemanha, República Democrática","3.376.290,00","5.902.230,00"
Angola,"821.790,00","1.263.525,00"
Anguilla,"0,00","0,00"
Antilhas Holandesas,"123.525,00","159.765,00"


In [7]:
# Países sem exportação
paises_sem_exportacao = total_por_pais.query('qtd_litros == 0 & vlr_usd == 0')


'''Obtendo a lista de paises com exportação à partir dos conjutos de índices
   dos dois dataframes - total_por_pais e paises_sem_exportacao'''

conj_paises = set(total_por_pais.index.tolist())
conj_paises_sem_exportacao = set(paises_sem_exportacao.index.tolist())

conj_paises_com_exportacao = conj_paises - conj_paises_sem_exportacao


# Países com exportação
paises_com_exportacao = total_por_pais.loc[list(conj_paises_com_exportacao)]
paises_com_exportacao['usd_por_litro'] = paises_com_exportacao['vlr_usd']\
                                         / paises_com_exportacao['qtd_litros']

print(f'''
Período de análise.............: {ano_inicial} - {ano_final}
Total de países................: {len(conj_paises)} países
Total de países com exportação.: {len(conj_paises_com_exportacao)} países
''')


Período de análise.............: 2009 - 2023
Total de países................: 137 países
Total de países com exportação.: 39 países



## Cinco países com maior volume total (em litros) das exportações

In [8]:
maiores_volumes = paises_com_exportacao.sort_values('qtd_litros',
                                                    ascending=False).head()

print(f'Dados do período total: {ano_inicial} - {ano_final}')
display(maiores_volumes)

Dados do período total: 2009 - 2023


,qtd_litros,vlr_usd,usd_por_litro
pais,,,
Rússia,"328.693.710,00","85.984.200,00","0,26"
China,"23.301.240,00","7.236.000,00","0,31"
Paraguai,"7.303.905,00","5.881.305,00","0,81"
Estados Unidos,"5.584.785,00","9.900.990,00","1,77"
"Alemanha, República Democrática","3.376.290,00","5.902.230,00","1,75"


O tabela acima demonstra, para o período total analisado os maiores volumes exportados, bem como a variação do valor total em dólares e do valor, em dólar, por litro para cada um dos países.

## Enriquecimento dos dados

In [9]:
# Dataframe com dados apenas dos países que possuem exportações
exportacao_analise = exportacao[exportacao['pais']\
                     .isin(list(conj_paises_com_exportacao))].copy()
exportacao_analise['usd_por_litro'] = exportacao_analise['vlr_usd']\
                                      / exportacao_analise['qtd_litros']

exportacao_analise.head()

,pais,qtd_litros,vlr_usd,ano,usd_por_litro
2,"Alemanha, República Democrática","225.086,00","393.482,00",2021,"1,75"
3,Angola,"54.786,00","84.235,00",2021,"1,54"
6,Antilhas Holandesas,"8.235,00","10.651,00",2021,"1,29"
8,Argentina,"162,00","4.523,00",2021,"27,92"
10,Austrália,"1.014,00","9.195,00",2021,"9,07"


In [10]:
# Incluindo o código ISO dos países

pais_cod_iso = pd.read_csv('dados/pais-cod-iso.csv')
display(pais_cod_iso.head())

exportacao_analise = pd.merge(exportacao_analise, pais_cod_iso,
                              left_on='pais', right_on='pais',
                              how='left')
display(exportacao_analise.head())

,pais,cod_iso
0,Afeganistão,AF
1,África do Sul,ZA
2,"Alemanha, República Democrática",DE
3,Angola,AO
4,Anguilla,AI


,pais,qtd_litros,vlr_usd,ano,usd_por_litro,cod_iso
0,"Alemanha, República Democrática","225.086,00","393.482,00",2021,"1,75",DE
1,Angola,"54.786,00","84.235,00",2021,"1,54",AO
2,Antilhas Holandesas,"8.235,00","10.651,00",2021,"1,29",AN
3,Argentina,"162,00","4.523,00",2021,"27,92",AR
4,Austrália,"1.014,00","9.195,00",2021,"9,07",AU
